# Running Tune experiments with BOHB

In this tutorial we introduce BOHB, while running a simple Ray Tune experiment. Tune’s Search Algorithms integrate with BOHB and, as a result, allow you to seamlessly scale up a BOHB optimization process - without sacrificing performance.

Bayesian Optimization HyperBand (BOHB) combines the benefits of Bayesian optimization together with Bandit-based methods (e.g. HyperBand). BOHB does not rely on the gradient of the objective function, but instead, learns from samples of the search space. It is suitable for optimizing functions that are nondifferentiable, with many local minima, or even unknown but only testable. Therefore, this approach belongs to the domain of "derivative-free optimization" and "black-box optimization".

In this example we minimize a simple objective to briefly demonstrate the usage of BOHB with Ray Tune via `BOHBSearch`. It's useful to keep in mind that despite the emphasis on machine learning experiments, Ray Tune optimizes any implicit or explicit objective. Here we assume `ConfigSpace==0.4.18` and `hpbandster==0.7.4` libraries are installed. To learn more, please refer to the [BOHB website](https://github.com/automl/HpBandSter).

In [ ]:
# !pip install ray[tune]
!pip install ConfigSpace==0.4.18
!pip install hpbandster==0.7.4

Click below to see all the imports we need for this example.
You can also launch directly into a Binder instance to run this notebook yourself.
Just click on the rocket symbol at the top of the navigation.

In [ ]:
import time

import ray
from ray import tune
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB
import ConfigSpace as CS

Let's start by defining a simple evaluation function.
We artificially sleep for a bit (`0.1` seconds) to simulate a long-running ML experiment.
This setup assumes that we're running multiple `step`s of an experiment and try to tune two hyperparameters,
namely `width` and `height`, and `activation`.

In [ ]:
def evaluate(step, width, height, activation):
    time.sleep(0.1)
    activation_boost = 10 if activation=="relu" else 1
    return (0.1 + width * step / 100) ** (-1) + height * 0.1 + activation_boost

Next, our `objective` function takes a Tune `config`, evaluates the `score` of your experiment in a training loop,
and uses `tune.report` to report the `score` back to Tune.

In [ ]:
def objective(config):
    for step in range(config["steps"]):
        score = evaluate(step, config["width"], config["height"], config["activation"])
        tune.report(iterations=step, mean_loss=score)

In [ ]:
ray.init(configure_logging=False)

Next we define a search space. The critical assumption is that the optimal hyperparameters live within this space. Yet, if the space is very large, then those hyperparameters may be difficult to find in a short amount of time.

In [ ]:
search_space = {
    "steps": 100,
    "width": tune.uniform(0, 20),
    "height": tune.uniform(-100, 100),
    "activation": tune.choice(["relu", "tanh"]),
}

Next we define the search algorithm built from `TuneBOHB`, constrained  to a maximum of `4` concurrent trials with a `ConcurrencyLimiter`. Below `algo` will take care of the BO (Bayesian optimization) part of BOHB, while scheduler will take care the HB (HyperBand) part.

In [ ]:
algo = TuneBOHB()
algo = tune.suggest.ConcurrencyLimiter(algo, max_concurrent=4)
scheduler = HyperBandForBOHB(
    time_attr="training_iteration",
    max_t=100,
    reduction_factor=4,
    stop_last_trials=False,
)

The number of samples is the number of hyperparameter combinations that will be tried out. This Tune run is set to `1000` samples.
(you can decrease this if it takes too long on your machine).

In [ ]:
num_samples = 1000

In [ ]:
# If 1000 samples take too long, you can reduce this number.
# We override this number here for our smoke tests.
num_samples = 10

Finally, we run the experiment to `"min"`imize the "mean_loss" of the `objective` by searching within `"steps": 100` via `algo`, `num_samples` times. This previous sentence is fully characterizes the search problem we aim to solve. With this in mind, notice how efficient it is to execute `tune.run()`.

In [ ]:
analysis = tune.run(
    objective,
    search_alg=algo,
    scheduler=scheduler,
    metric="mean_loss",
    mode="min",
    name="bohb_exp",
    num_samples=num_samples,
    config=search_space
)

Here are the hyperparameters found to minimize the mean loss of the defined objective.

In [ ]:
print("Best hyperparameters found were: ", analysis.best_config)

## Optional: Passing the search space via the TuneBOHB algorithm

We can define the hyperparameter search space using `ConfigSpace`, which is the format accepted by BOHB.

In [ ]:
config_space = CS.ConfigurationSpace()
config_space.add_hyperparameter(
    CS.UniformFloatHyperparameter("width", lower=0, upper=20)
)
config_space.add_hyperparameter(
    CS.UniformFloatHyperparameter("height", lower=-100, upper=100)
)
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        "activation", choices=["relu", "tanh"]
    )
)

In [ ]:
algo = TuneBOHB(
    space=config_space,
    metric="episode_reward_mean",
    mode="max",
)
algo = tune.suggest.ConcurrencyLimiter(algo, max_concurrent=4)
scheduler = HyperBandForBOHB(
    time_attr="training_iteration",
    max_t=100,
    reduction_factor=4,
    stop_last_trials=False,
)

analysis = tune.run(
    objective,
    config=config_space,
    scheduler=bohb_hyperband,
    search_alg=algo,
    num_samples=num_samples,
    name="bohb_exp_2",
    stop={"training_iteration": 100},
)

Here again are the hyperparameters found to minimize the mean loss of the defined objective.

In [ ]:
print("Best hyperparameters found were: ", analysis.best_config)

In [ ]:
ray.shutdown()